# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
## Part 3: Clustering the neighborhoods in Toronto
### Install and import necessary packages

In [1]:
!pip install bs4
!pip install geopy
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import json
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Set up the dataframes as they were at the end of Part 2

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
table = BeautifulSoup(html.text, 'html.parser').find('table')  #Finds the first table in the page
table_rows = table.find_all('tr')                              #Finds all <tr> tags and returns a list of them
data = []
for row in table_rows:                               #Iterate through all rows
    tds = row.find_all('td')                         #Find all <td> tags
    new_row = [td.text.strip() for td in tds]        #Create a list of the td elements, while removing the "\n" from the end
    if len(new_row):
        data.append(new_row)
df = pd.DataFrame(data, columns=["Postal code", "Borough", "Neighborhood"])
df = df.drop(df[df["Borough"] == "Not assigned"].index)
df["Neighborhood"] = df["Neighborhood"].str.replace(" / ", ", ")
df["Neighborhood"] = df["Neighborhood"].str.replace("\\n", "")
df.reset_index(inplace = True, drop = True)
coords = pd.read_csv("https://cocl.us/Geospatial_data")
toronto = df.join(other=coords.set_index("Postal Code"), on="Postal code")

In [3]:
print(toronto.shape)
toronto.head()

(103, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Using the geopy library we determine the coordinates of Toronto

In [4]:
geolocator = Nominatim(user_agent="toronto_cluster")
location = geolocator.geocode("Toronto, Ontario, Canada")
toronto_lat = location.latitude
toronto_long = location.longitude
print(toronto_lat, toronto_long)

43.6534817 -79.3839347


### Visualize Toronto's neighborhoods on a map

In [5]:
map = folium.Map(location=[toronto_lat, toronto_long], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
map

### Set up Fourquare credentials

In [6]:
CLIENT_ID = 'ZBC2EWETE1R2DGRBAWJ1YGOOUBCTXCOZTJS4JHQ0V1MBJPJU' # your Foursquare ID
CLIENT_SECRET = 'UGTY5QSD0CVQQC1DFIKRZULRMNVZJSFAPKVHWKSZXR0ADP5Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

### Define a function to get the category of the resulted venue. If more than one categories are given, then the function returns only the first one.

In [7]:
def get_category(row):
    category_list = row["venue.categories"]
    if len(category_list):
        return category_list[0]["name"]
    else:
        return None

### Let's explore the first neighborhood, Parkwoods.

In [9]:
radius = 700

latitude = toronto.iloc[0,3]
longitude = toronto.iloc[0,4]

url = url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            latitude,
            longitude,
            radius,
            LIMIT)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
df_venues = pd.json_normalize(venues)
df_venues = df_venues[["venue.name", "venue.location.lat", "venue.location.lng", "venue.categories"]]
df_venues["venue.categories"] = df_venues.apply(get_category, axis=1)
df_venues.columns = [col.split('.')[-1].capitalize() for col in df_venues.columns]
print(df_venues.shape)
df_venues

(5, 4)


,Name,Lat,Lng,Categories
0,Brookbanks Park,43.751976,-79.332140,Park
1,PetSmart,43.748639,-79.333488,Pet Store
2,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Joey,43.753441,-79.321640,Burger Joint
4,Three Valleys Park,43.751195,-79.337356,Park


With a radius of 500 meters, there are only 2 results in Parkwoods. Even with an extended radius of 700 we have found only 5 venues. So in the code below, I will extend the radius with 200 meters if the number of results is less than 10, but it is maximised at 900.

I create an empty dataframe, then append each venue one-by-one. The resulted dataframe has only 5 columns as visible below.

In [10]:
toronto_venues = pd.DataFrame(columns=["Neighborhood", "Venue name", "Venue Latitude", "Venue Longitude", "Category"])

def set_url(lat, lng, radius):
    return 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

for i, row in toronto.iterrows():
    #time.sleep(0.1)
    radius = 500
    lat = row["Latitude"]
    lng = row["Longitude"]
    nbhood = row["Neighborhood"]
    print(i, nbhood, "...", end='')
    while True:
        url = set_url(lat, lng, radius)
        results = requests.get(url).json()
        venues = results['response']['groups'][0]['items']
        if len(venues) >= 10 or radius >= 900:
            break
        radius += 200
    if len(venues) == 0:
        print("could not find any venues in {}!".format(nbhood))
        continue
    df_venues = pd.json_normalize(venues)
    df_venues = df_venues[["venue.name", "venue.location.lat", "venue.location.lng", "venue.categories"]]
    df_venues["venue.categories"] = df_venues.apply(get_category, axis=1)
    for j, venue in df_venues.iterrows():
        toronto_venues = toronto_venues.append({
            "Neighborhood": nbhood,
            "Venue name": df_venues.loc[j, "venue.name"],
            "Venue Latitude": df_venues.loc[j, "venue.location.lat"],
            "Venue Longitude": df_venues.loc[j, "venue.location.lng"],
            "Category": df_venues.loc[j, "venue.categories"]
        }, ignore_index=True)
    print("  {} venues found!".format(len(venues)))
    
print(toronto_venues.shape)
toronto_venues.head()

0 Parkwoods ...  17 venues found!
1 Victoria Village ...  10 venues found!
2 Regent Park, Harbourfront ...  47 venues found!
3 Lawrence Manor, Lawrence Heights ...  14 venues found!
4 Queen's Park, Ontario Provincial Government ...  31 venues found!
5 Islington Avenue ...  10 venues found!
6 Malvern, Rouge ...  17 venues found!
7 Don Mills ...  13 venues found!
8 Parkview Hill, Woodbine Gardens ...  13 venues found!
9 Garden District, Ryerson ...  100 venues found!
10 Glencairn ...  18 venues found!
11 West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale ...  17 venues found!
12 Rouge Hill, Port Union, Highland Creek ...  4 venues found!
13 Don Mills ...  23 venues found!
14 Woodbine Heights ...  25 venues found!
15 St. James Town ...  86 venues found!
16 Humewood-Cedarvale ...  25 venues found!
17 Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood ...  18 venues found!
18 Guildwood, Morningside, West Hill ...  22 venues found!
19 The Beaches ...  41 venues

,Neighborhood,Venue name,Venue Latitude,Venue Longitude,Category
0,Parkwoods,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,Bruno's valu-mart,43.746143,-79.324630,Grocery Store


Cool! So, we have found 2738 venues in Toronto. But there are not any venues in Upper Rouge. That may cause problems later!

And how many categories?

In [11]:
print('There are {} unique categories.'.format(len(toronto_venues["Category"].unique())))

There are 284 unique categories.


I generate one-hot-encoding

In [12]:
toronto_onehot = pd.get_dummies(toronto_venues[["Category"]], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant

### Wow! How is it possible that Yoga Studio comes to the first place? 'Neighborhood' did not added? Seems so. But look at below: We still can group the results by neighborhood. Is this a bug in pandas? Just let me know if you have any idea :)

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Transportation Service,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant

In [14]:
toronto_grouped.shape

(97, 284)

### Hmm... We used to have 103 Neighborhoods. Now we have only 97. Why?

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(97, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Skating Rink,Mediterranean Restaurant,Bank,Sandwich Place,Sushi Restaurant,Supermarket,Caribbean Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Skating Rink,Sandwich Place,Pool,Coffee Shop,Pub,Athletics & Sports,Doner Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Gas Station,Bridal Shop,Sandwich Place,Middle Eastern Restaurant,Supermarket,Diner,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Gas Station,Café,Restaurant,Chinese Restaurant,Intersection,Shopping Mall,Park,Grocery Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Pizza Place,Pharmacy,Butcher,Café,Pub,Comfort Food Restaurant


In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Bus Stop,Coffee Shop,Road,Caribbean Restaurant,Supermarket,Chinese Restaurant,Food & Drink Shop,Laundry Service,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Park,Playground,Sporting Goods Shop,Café,Portuguese Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Intersection,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Health Food Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Gift Shop,Eastern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Diner,Fried Chicken Joint,Discount Store,Distribution Center,Bank,Bar,Italian Restaurant,Yoga Studio,Theater
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,4.0,Pharmacy,Golf Course,Park,Café,Shopping Mall,Skating Rink,Bank,Playground,Grocery Store,Golf Driving Range
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Trail,Martial Arts Dojo,Bus Station,Gym,Paper / Office Supplies Store,Construction & Landscaping,Bakery,Hobby Shop,Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Supermarket,Restaurant,Café,Sporting Goods Shop,Chinese Restaurant
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0.0,Pizza Place,Athletics & Sports,Pharmacy,Bus Line,Café,Bank,Breakfast Spot,Gastropub,Intersection,Fast Food Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Fast Food Restaurant


Upper Rouge has a label of NaN. We were expecting some trouble here. But how does it come that we could join the 97 rows toronto_grouped table with the 103 row toronto without any problem; and 102 out of the 103 have their most common categories?

In order not to delete upper rouge, I will give it a unique label, I will assign it to a 6th cluster by hand.

In [19]:
toronto_merged.iloc[95, 5] = 5.0

And we turn the label values back to integer

In [20]:
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)
toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Park,Bus Stop,Coffee Shop,Road,Caribbean Restaurant,Supermarket,Chinese Restaurant,Food & Drink Shop,Laundry Service,Discount Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Park,Playground,Sporting Goods Shop,Café,Portuguese Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Intersection,Ethiopian Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Health Food Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Gift Shop,Eastern European Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Fried Chicken Joint,Discount Store,Distribution Center,Bank,Bar,Italian Restaurant,Yoga Studio,Theater


In [21]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's see the clusters one-by-one
#### Cluster 1

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Parkview Hill, Woodbine Gardens",Pizza Place,Athletics & Sports,Pharmacy,Bus Line,Café,Bank,Breakfast Spot,Gastropub,Intersection,Fast Food Restaurant
10,Glencairn,Pizza Place,Grocery Store,Park,Coffee Shop,Japanese Restaurant,Rental Car Location,Ice Cream Shop,Sandwich Place,Gas Station,Pub
18,"Guildwood, Morningside, West Hill",Park,Fast Food Restaurant,Pizza Place,Sports Bar,Breakfast Spot,Bus Line,Restaurant,Rental Car Location,Pharmacy,Moving Target
27,Hillcrest Village,Park,Pharmacy,Pizza Place,Bakery,Ice Cream Shop,Korean Restaurant,Coffee Shop,Pool,Chinese Restaurant,Recreation Center
28,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Gas Station,Bridal Shop,Sandwich Place,Middle Eastern Restaurant,Supermarket,Diner,Sushi Restaurant
29,Thorncliffe Park,Indian Restaurant,Yoga Studio,Bank,Gas Station,Liquor Store,Fast Food Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Gym
31,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Pizza Place,Park,Music Venue,Café,Middle Eastern Restaurant,Brewery,Supermarket,Bar
32,Scarborough Village,Ice Cream Shop,Pizza Place,Coffee Shop,Sandwich Place,Restaurant,Train Station,Fast Food Restaurant,Convenience Store,Discount Store,Farmers Market
34,"Northwood Park, York University",Pizza Place,Frame Store,Massage Studio,Caribbean Restaurant,Bank,Bar,Coffee Shop,Falafel Restaurant,Fast Food Restaurant,Japanese Restaurant
39,Bayview Village,Japanese Restaurant,Bank,Gas Station,Café,Restaurant,Chinese Restaurant,Intersection,Shopping Mall,Park,Grocery Store


#### Cluster 2

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Park,Bus Stop,Coffee Shop,Road,Caribbean Restaurant,Supermarket,Chinese Restaurant,Food & Drink Shop,Laundry Service,Discount Store
1,Victoria Village,Park,Playground,Sporting Goods Shop,Café,Portuguese Restaurant,Hockey Arena,Coffee Shop,Pizza Place,Intersection,Ethiopian Restaurant
2,"Regent Park, Harbourfront",Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Health Food Store
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,Accessories Store,Boutique,Vietnamese Restaurant,Event Space,Miscellaneous Shop,Coffee Shop,Gift Shop,Eastern European Restaurant
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Fried Chicken Joint,Discount Store,Distribution Center,Bank,Bar,Italian Restaurant,Yoga Studio,Theater
6,"Malvern, Rouge",Fast Food Restaurant,Trail,Martial Arts Dojo,Bus Station,Gym,Paper / Office Supplies Store,Construction & Landscaping,Bakery,Hobby Shop,Restaurant
7,Don Mills,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Supermarket,Restaurant,Café,Sporting Goods Shop,Chinese Restaurant
9,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Diner,Fast Food Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",Pizza Place,Park,Hotel,Bank,Mexican Restaurant,Restaurant,Clothing Store,Café,Coffee Shop,Fish & Chips Shop
12,"Rouge Hill, Port Union, Highland Creek",Playground,Breakfast Spot,Burger Joint,Italian Restaurant,Fast Food Restaurant,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 3

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Park,Baseball Field,Shopping Mall,Women's Store,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


#### Cluster 4

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Caledonia-Fairbanks,Park,Women's Store,Japanese Restaurant,Beer Store,Mexican Restaurant,Sporting Goods Shop,Bakery,Grocery Store,Gym,Event Space
22,Woburn,Park,Coffee Shop,Women's Store,Empanada Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
49,"North Park, Maple Leaf Park, Upwood Park",Gas Station,Bakery,Convenience Store,Park,Coffee Shop,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
66,York Mills West,Park,Tennis Court,Bank,Gym,Pet Store,French Restaurant,Convenience Store,Restaurant,Dog Run,Donut Shop
91,Rosedale,Park,Coffee Shop,Grocery Store,Playground,Trail,Candy Store,Sandwich Place,Japanese Restaurant,Bank,Athletics & Sports


#### Cluster 5

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Islington Avenue,Pharmacy,Golf Course,Park,Café,Shopping Mall,Skating Rink,Bank,Playground,Grocery Store,Golf Driving Range
62,Roselawn,Playground,Skating Rink,Garden,Café,Pharmacy,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Farmers Market


#### And finally our manually created cluster 6 with its only member: Upper Rouge

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Upper Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
